# Aula 5.3: Conhecendo as APIs do BACEN


## O Portal
O [Portal Brasileiro de Dados Abertos do Banco Central](https://dadosabertos.bcb.gov.br/dataset) é o meio utilizado pelo BC para disponibilizar dados e informações públicas. Ele foi criado para auxiliar os usuários a localizar os conjuntos de dados de seu interesse, entender a estrutura desses dados e encontrar o caminho para acessá-los. Uma vez localizado o dado e entendida a sua estrutura, o usuário poderá utilizar alguma ferramenta de análise ou algum software de programação para acessar os dados propriamente ditos. O Portal contém dados em sua forma bruta, sem formatações visuais, para facilitar o processamento por computadores.


## Dados Disponíveis
São dados armazenados em bases de dados mantidas pelo Banco Central do Brasil, desde que sobre elas não recaia hipótese de restrição de acesso.

Também estão disponíveis os Dados do Sistema Financeiro Nacional, constituídos por informações de interesse público não sujeitas a hipóteses de sigilo, disponibilizadas em formato aberto pelas instituições financeiras e demais instituições autorizadas a funcionar pelo BC, com o objetivo de promover a competitividade, transparência e inovação no setor financeiro.

In [ ]:
import requests                    # api module
import json
import pandas as pd

#Obtendo os dados
url = 'https://olinda.bcb.gov.br/olinda/servico/Informes_Agencias/versao/v1/odata/Agencias?$format=json&$select=Segmento,MunicipioIbge,Municipio,UF'
response = requests.get(url)
#bcbJson = response.json()
print(response)

É sempre bom checar se o retono da requisição foi 200 (OK).

In [ ]:
agencias = response.json()
print(agencias)

O resultado da requisição contém informações sobre instituições bancárias no Brasil.Vamos carregar o resultado da requisição em um Dataframe

In [ ]:
dfBancos = pd.json_normalize(agencias['value'])
dfBancos.head()

In [ ]:
dfBancos.describe()

Obseve que existem 21 categorias para segmento. Vamos calcular a quantidade de instituiçõe de cada tipo, nas cidades. Primeiro vamos recuperar uma lista com os tipos de instituição

In [ ]:
fullNameList = pd.unique(dfBancos['Segmento'])
initialList = []
for names in fullNameList:
  initialList.append(''.join([x[0] for x in names.split(' ')]))

instSigla = dict(zip(initialList, fullNameList))

Vamos recuperar as informações da API do IBGE:

In [ ]:
# Obtendo os dados per capita - ceará e pernambuco
url = 'https://servicodados.ibge.gov.br/api/v3/agregados/3974/periodos/2010/variaveis/3948?localidades=N6[N3[23,26]]&classificacao=12085[100543]|58[95253]'
response = requests.get(url)
pib = response.json()
for item in pib:
  for key in item['resultados']:
    pibJson = key
pibJson.pop('classificacoes')
dfPIB = pd.json_normalize(pibJson['series'])
dfPIB[['CIDADE','UF']] = dfPIB[dfPIB.columns[3]].str.split(' - ',1).tolist()
dfPIB.rename(columns = {dfPIB.columns[0]:'ID', 
                        dfPIB.columns[-3]: 'PIB'},
             inplace = True)
dfPIB.set_index('ID', inplace = True)

new_columns = (dfPIB.columns.drop('PIB').tolist()) + ['PIB']
dfPIB = dfPIB[new_columns]
dfPIB.drop(dfPIB.columns[0:3], axis=1,inplace=True)

# Obtendo a densidade populacional: Ceará e Pernambuco
url = "https://servicodados.ibge.gov.br/api/v3/agregados/1301/periodos/2010/variaveis/616?localidades=N6[N3[23,26]]"

response = requests.get(url)
dens = response.json()
for item in dens:
  for key in item['resultados']:
    densJson = key
densJson.pop('classificacoes')
dfDens = pd.json_normalize(densJson['series'])
dfDens.rename(columns = {dfDens.columns[0]:'ID', 
                         dfDens.columns[-1]: 'DENS'},
             inplace = True)
dfDens.set_index('ID', inplace = True)
dfDens.drop(dfDens.columns[0:3], axis=1,inplace=True)


# Obtendo os dados de escolarização por grupo - ceará e pernambuco
url = 'https://servicodados.ibge.gov.br/api/v3/agregados/3955/periodos/2010/variaveis/3930?localidades=N6[N3[23,26]]&classificacao=12085[100543]|58[95253]'
response = requests.get(url)
esc = response.json()
for item in esc:
  for key in item['resultados']:
    escJson = key
escJson.pop('classificacoes')
dfEsc = pd.json_normalize(escJson['series'])

dfEsc.rename(columns = {dfEsc.columns[0]:'ID',
                       dfEsc.columns[-1]: 'ESC'},
             inplace = True)
dfEsc.set_index('ID', inplace = True)
dfEsc.drop(dfEsc.columns[0:3], axis=1,inplace=True)

In [ ]:
dfPIB.head()

In [ ]:
dfDens.head()

In [ ]:
dfEsc.head()

Vamos criar um dataframe único:

In [ ]:
df = dfPIB.merge(dfDens.merge(dfEsc,left_index=True, right_index=True),left_index=True, right_index=True)
df.head()

Ótimo. Agora vamos retornar ao dataframe dos dados bancários. Iremos recuperar apenas os dados dos estados CE e PE:

In [ ]:
dfBancos = pd.concat([dfBancos[dfBancos['UF']=='CE'],dfBancos[dfBancos['UF']=='PE']])
dfBancos.head()

Vamos utilizar o método `value_counts` para verificar a quantidade de amostras por segmento:

In [ ]:
dfBancos.value_counts(subset = 'Segmento')

In [ ]:
dfBancos.describe()

Certo, temos apenas 9 classes de segmento para CE e PE. Vamos criar uma lista de abreviaturas para adicionar ao dataframe que já contem os dados de PIB, Densidade Populacional e Escolaridade:

In [ ]:
fullNameList = pd.unique(dfBancos['Segmento'])
initialList = []
for names in fullNameList:
  initialList.append(''.join([x[0] for x in names.split(' ')]))

`fullNameList`: Corresponde às classes dos Segmentos

`initialList`: Corresponde às iniciais dos Segmentos

Agora, precisamos contar quantas vezes cada classe é observada em cada cidade. Quantos Bancos Múltiplos existem na cidade de Juazeiro do Norte-CE? Para isso, vamos usar o método `groupby`:

In [ ]:
dfTest = dfBancos
for i in range(0,len(initialList)):
  dfTest[initialList[i]] = dfTest[ dfTest['Segmento']== fullNameList[i]].groupby(['MunicipioIbge'])['Segmento'].transform('count')

O que fizemos aqui? Buscamos cada classe de segmento no DataFrame e agrupamos po Município. Por fim, usamos `transform('count')` para contabilizar a quantidade de itens. Adicionamos essas contagens a novas colunas das iniciais de cada categoria.

In [ ]:
dfTest.head()

O processo acabou linhas colunas duplicadas. Vamos remover os segmentos duplicados em cada município:

In [ ]:
dfTest = dfTest.drop_duplicates(subset=['Segmento','MunicipioIbge'])
dfTest.head()

Ótimo. Porém, agora temos NaN nas colunas que não puderam ser preenchidas. Vamos usar o `fillna(0)` para lidar com esse problema.

In [ ]:
dfTest=dfTest.fillna(0)
dfTest.head()

Vamos renomear as colunas `Segmento`,  `MunicipioIbge` e  `Municipio` para padronizar com o DataFrame do IBGE:

In [ ]:
dfTest.rename(columns = {dfTest.columns[1]:'ID',
                       dfTest.columns[2]: 'CIDADE'},
             inplace = True)
dfTest.drop(['Segmento'], axis=1,inplace=True)
dfTest.set_index('ID', inplace = True)
dfTest.head()

Agora precisamos usar o `groupby` novamente para juntar linhas da mesma cidade:

In [ ]:
dfTest = dfTest.groupby(['ID'])[initialList].sum()
dfTest.head()

Conseguimos construir uma tabela que relaciona as classes (itens) dos segmentos à quantidade em cada cidade. Vamos verificar as intituições na cidade de Fortaleza:

In [ ]:
fullNameList

In [ ]:
dfTest[dfTest.index == '2304400']

Após esse pré-processamento, vamos concatenar os datasets do IBGE e do Banco Central:

In [ ]:
df = df.merge(dfTest,left_index=True, right_index=True, how='outer').fillna(0)
df.head()

In [ ]:
df.describe()

Para Clustering, podemos começar pelas colunas a seguir:

In [ ]:
print(df.columns[2:].tolist())

Vamos escolher as cinco primeiras classes para clusterização:

In [ ]:
cols = df.columns[2:8].tolist()
X = df[cols].to_numpy()
print(X)

In [ ]:
df[cols]=df[cols].astype(float)
X = df[cols].to_numpy()
print(X)

Executando o KMeans com 8 clusters:

In [ ]:
from sklearn.cluster import KMeans
cl = KMeans(n_clusters=8, random_state=0).fit(X)
clusters = pd.DataFrame(cl.labels_, columns=['cluster'])
detail = pd.merge(df.reset_index(), clusters, right_index=True, left_index=True)
detail['cluster'] = detail['cluster'].astype(str)
detail.head()

Vamos obter novamente os shapes:

In [ ]:
#Shapes CE
url = 'https://servicodados.ibge.gov.br/api/v3/malhas/estados/23?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio'
response = requests.get(url)
shapesJsonCE = response.json()


#Shapes PE
url = 'https://servicodados.ibge.gov.br/api/v3/malhas/estados/26?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio'
response = requests.get(url)
shapesJsonPE = response.json()
shapesJson = shapesJsonCE
shapesJson['features'] += shapesJsonPE['features']

In [ ]:
import plotly.express as px
from plotly.offline import init_notebook_mode, plot, iplot, download_plotlyjs
br_lat = -6
br_lon = -38.0
brazilMap = px.choropleth_mapbox(detail,
                geojson=shapesJson, 
                locations='ID',
                hover_name = 'CIDADE',
                color="cluster", 
                featureidkey="properties.codarea",
                )
brazilMap.update_layout(mapbox_style="open-street-map", 
                            mapbox_zoom=5, 
                            mapbox_center = {"lat": br_lat, "lon": br_lon},
                            title="Clusters - Kmeans")
brazilMap.show()